In [3]:
!pip3 install pandas dash


     |████████████████████████████████| 9.9 MB 122 kB/s eta 0:00:01
     |████████████████████████████████| 421 kB 74 kB/s eta 0:00:011


In [7]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})
# Create a dash application
app = dash.Dash(__name__)
                               
app.layout = html.Div(children=[ html.H1('Airline Performance Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 40}),
                                html.Div(["Input Year: ", dcc.Input(id='input-year', value='2010', 
                                type='number', style={'height':'50px', 'font-size': 35}),], 
                                style={'font-size': 40}),
                                html.Br(),
                                html.Br(),
                                html.Div(dcc.Graph(id='line-plot')),
                                ])

# add callback decorator
@app.callback( Output(component_id='line-plot', component_property='figure'),
               Input(component_id='input-year', component_property='value'))

# Add computation to callback function and return graph
def get_graph(entered_year):
    # Select 2019 data
    df =  airline_data[airline_data['Year']==int(entered_year)]
    
    # Group the data by Month and compute average over arrival delay time.
    line_data = df.groupby('Month')['ArrDelay'].mean().reset_index()

    fig = go.Figure(data=go.Scatter(x=line_data['Month'], y=line_data['ArrDelay'], mode='lines', marker=dict(color='green')))
    fig.update_layout(title='Month vs Average Flight Delay Time', xaxis_title='Month', yaxis_title='ArrDelay')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Aug/2022 18:34:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2022 18:34:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2022 18:34:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2022 18:34:45] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2022 18:34:45] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2022 18:34:45] "POST /_dash-update-component HTTP/1.1" 200 -


In [1]:
import wget
spacex_launch_dash = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")


100% [............................................................] 2476 / 2476

In [58]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

op = [{'label': 'All Sites', 'value': 'ALL'}]
for i in spacex_df['Launch Site'].unique():
    op.append({'label': i, 'value': i})

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=op,
                                             value='ALL', # This one is defult inactivating this will show placeholder
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                            ),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                #marks={0: '0', 100: '100'},
                                                value=[min_payload, max_payload]
                                               ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.pie(spacex_df[spacex_df['Launch Site']==entered_site], names='class',   title='Success/Unsuccess Launches (Percentage) for Site '+ entered_site)
        return fig
        
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_chart(entered_site,slider_range):
    filtered_df = spacex_df.loc[(spacex_df['Payload Mass (kg)']>slider_range[0]) & (spacex_df['Payload Mass (kg)']<slider_range[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, y='class', x = 'Payload Mass (kg)', color='Booster Version Category',   title='Correlation between Success/Unsuccess event for All Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.scatter(filtered_df[filtered_df['Launch Site']==entered_site], x = 'Payload Mass (kg)', color='Booster Version Category',  title='Correlation between Success/Unsuccess event for Site '+ entered_site)
        return fig



# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Envi

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Sep/2022 10:58:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:03] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:03] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:03] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:03] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:03] "GET /_favicon.ico?v=2.6.1 HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2022 10:58:06] "POST /_dash-upd

In [25]:
px.pie(spacex_df[spacex_df['Launch Site']=='CCAFS LC-40'], names='class',   title='title')

In [9]:
px.pie(spacex_df, values='class', names='Launch Site',   title='title')

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0
13,13,15,CCAFS LC-40,0,570.0,F9 v1.1 B1013,v1.1
26,26,6,VAFB SLC-4E,0,500.0,F9 v1.1 B1003,v1.1
27,27,21,VAFB SLC-4E,0,553.0,F9 v1.1 B1017,v1.1
30,30,40,VAFB SLC-4E,1,475.0,F9 FT B1038.1,FT
54,54,53,CCAFS SLC-40,1,362.0,F9 B4 B1045.1,B4


In [29]:
'Total Success Launches for Site'+ ' entered_site'

'Total Success Launches for Site entered_site'